<a href="https://colab.research.google.com/github/PapaZeusOne/Automated-Trading-Bot/blob/main/Automating_Stock_Trades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [152]:
import numpy as np 
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
from datetime import date, timedelta


In [153]:
def calc_indicators(df):
  df['ema_25'], df['ema_100'] = calc_ema(df)
  
  df['sma_25'], df['sma_100'] = calc_sma(df)

  df['bollinger'], df['bollinger_lower_band'], df['bollinger_upper_band'] = calc_bollinger(df)

  df['macd'], df['signalline'] = calc_macd_signalline(df)

  df['RSI'] = calc_rsi(df)

  df['CCI'] = calc_cci(df)

  # df['OBV_MA'] = calc_obv(df)
   
  df = calculate_dead_cat_bounce_v2(df)

  return df

In [154]:
def calc_ema(df):
    ema_25 = 25
    df_ema_25 = df['Adj Close'].ewm(span=ema_25, adjust=False).mean()

    ema_100 = 100
    df_ema_100 = df['Adj Close'].ewm(span=ema_100, adjust=False).mean()

    return df_ema_25, df_ema_100

In [155]:
def calc_sma(df):
    sma_25 = 25
    df_sma_25 = df['Adj Close'].rolling(window=sma_25).mean()

    sma_100 = 100
    df_sma_100 = df['Adj Close'].rolling(window=sma_100).mean()

    return df_sma_25, df_sma_100

In [156]:
def calc_bollinger(df):
    bollinger_avg = 20

    short_avg_ma = df['Adj Close'].rolling(window=bollinger_avg).mean()
    short_avg_std = df['Adj Close'].rolling(window=bollinger_avg).std()

    bollinger = df['Adj Close'].ewm(span=bollinger_avg, adjust=False).mean()
    bollinger_lower_band = short_avg_ma - (short_avg_std *2)
    bollinger_upper_band = short_avg_ma + (short_avg_std *2)

    return bollinger, bollinger_lower_band, bollinger_upper_band

In [157]:
def calc_macd_signalline(df):
    ema_short = 12
    ema_long = 26
    signalline_span = 9

    df_ema_short = df['Adj Close'].ewm(span=ema_short, adjust=False).mean()
    df_ema_long = df['Adj Close'].ewm(span=ema_long, adjust=False).mean()
    macd = df_ema_short - df_ema_long
    signalline = macd.ewm(span=signalline_span, adjust=False).mean()

    return macd, signalline

In [158]:
def calc_rsi(df, n=14):
    delta = df['Adj Close'].diff()
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    # Calculate the EWMA
    RolUp = dUp.ewm(span=n).mean()
    RolDown = dDown.abs().ewm(span=n).mean()

    # Calculate the RSI based on EWMA
    RS = RolUp / RolDown
    RSI = 100 - (100/(1+RS))

    # Calculate the SMA
    roll_up2 = dUp.rolling(n).mean()
    roll_down2 = dDown.abs().rolling(n).mean()

    # Calculate the RSI based on SMA
    RS2 = roll_up2 / roll_down2
    RSI2 = 100.0 - (100.0 / (1.0 + RS2))

    own_RSI=(RSI+RSI2)/2
    return own_RSI

In [159]:
def calc_cci(df, n=14, constant=0.015):
    tippingPrice = (df['High'] + df['Low'] + df['Adj Close']) / 3
    tippingPrice_mean = tippingPrice.rolling(n).mean()
    tippingPrice_std = tippingPrice.rolling(n).std()
    
    CCI = pd.Series((tippingPrice - tippingPrice_mean) / (constant * tippingPrice_std)) 
    return CCI

In [160]:
### To Be Adjusted

def calc_obv(df, n = 20):
    i = 0
    OBV = [0]

    while i < df.index[-1]:
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] > 0:
            OBV.append(df.loc[i + 1, 'Volume'])
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] == 0:
            OBV.append(0)
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] < 0:
            OBV.append(-df.loc[i + 1, 'Volume'])
        i = i + 1
    OBV = pd.Series(OBV)
    OBV_ma = pd.Series(OBV.rolling(n, min_periods=n).mean())

    return OBV_ma

In [161]:
def calculate_dead_cat_bounce_v2(df):
  set_freq = '2D'

  df['body_mid'] = (df['Adj Close'] + df['Open'])/2

  pct_change_close = df['body_mid'].pct_change(freq=set_freq, fill_method='ffill')
  pct_change_close.dropna(inplace=True)

  dead_cat_bounce = np.where((df.signalline > df.macd) &
                            ((pct_change_close >= 0.05) | (pct_change_close <= -0.05)) &
                            ((df.Open < df.bollinger_lower_band)|(df.Close < df.bollinger_lower_band)),
                            True, False)

  df['dead_cat_bounce'] = dead_cat_bounce
  del df['body_mid']

  return df

In [162]:
def combine_stocks_today(id_list):

  ts = TimeSeries(key='AXUYKPP8SFQ1MAB7', output_format='pandas')  
  start_date = '2020-7-28'
  df_list = []

  for id in id_list:
    data, meta_data = ts.get_daily_adjusted(symbol=id, outputsize='full') # Change outputsize to 'compact' and only receive data from the last 100 days
    data = data.sort_index(ascending=True)
    data['stock_id'] = id
    
    del data['6. volume']
    del data['7. dividend amount']
    del data['8. split coefficient']

    # Resize df and rename columns appropriately
    stock = data[start_date:].copy()
    stock.columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'stock_id']

    # calc indicators for every stock
    stock = calc_indicators(stock) 

    # drop unnecessary rows with NaN values
    stock.dropna(inplace=True) 
    df_list.append(stock)

  df_complete = pd.concat(df_list)
  
  return df_complete

In [163]:
def combine_stocks_14d_ago(id_list):

  ts = TimeSeries(key='AXUYKPP8SFQ1MAB7', output_format='pandas')  
  start_date = '2020-7-28'
  df_list = []

  for id in id_list:
    data, meta_data = ts.get_daily_adjusted(symbol=id, outputsize='full') # Change outputsize to 'compact' and only receive data from the last 100 days
    data = data.sort_index(ascending=True)
    data['stock_id'] = id
    
    del data['6. volume']
    del data['7. dividend amount']
    del data['8. split coefficient']

    # Resize df and rename columns appropriately
    date = date.today()
    date_15d_ago = date + timedelta(days=-15)
    stock = data[start_date:date_15d_ago].copy()
    stock.columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'stock_id']

    # calc indicators for every stock
    stock = calc_indicators(stock) 

    # drop unnecessary rows with NaN values
    stock.dropna(inplace=True) 
    df_list.append(stock)

  df_complete = pd.concat(df_list)
  
  return df_complete

<br> Combine Stocks and calculate the values for each given indicator </br>

In [164]:
stocks_dax = ['ADS.DE', 'ALV.DE', 'BAS.DE', 'BAYN.DE', 'BMW.DE', 'CON.DE', '1COV.DE', 'DAI.DE', 'DHER.DE', 'DBK.DE', 'DB1.DE', 'DPW.DE', 'DTE.DE', 'DWNI.DE', 'EOAN.DE', 'FRE.DE', 'FMS', 'HEI.DE', 'HEN3.DE', 'IFX.DE', 'LIN.DE', 'MRK', 'MTX.DE', 'MUV2.DE', 'RWE.DE', 'SAP.DE', 'SIE.DE', 'ENR.F', 'VOW.DE', 'VNA.DE']
stocks_random = ['AMZN', 'AAPL', 'SAP', 'ACN', 'VOW.DE', 'PLUG', 'MDO.DE', 'PYPL', 'V', 'YNDX']

stock_list_1 = ['AMZN', 'AAPL', 'SAP', 'ACN', 'VOW.DE']
stock_list_2 = ['PLUG', 'MDO.DE', 'PYPL', 'V', 'YNDX']

#stocks_combine = np.concatenate([stocks_dax, stocks_random])
df_complete = combine_stocks_today(stock_list_1)

In [165]:
df_yd = df_complete.drop_duplicates(subset='stock_id', keep='last')

<br> Calculate a score based on the indicators </br>

In [166]:
def ema_score(row):
    ema_25 = row['ema_25']
    ema_100 = row['ema_100']
    score = (ema_25/ema_100)-1
    return score

In [167]:
def sma_score(row):
    sma_25 = row['sma_25']
    adj_close = row['Adj Close']
    score = (adj_close/sma_25)-1
    return score

In [168]:
def macd_signalline_score(row): 
    # relative change might be distorted if values near by 0
    macd = row['macd']
    signalline = row['signalline']
    adj_close = row['Adj Close']
    score = (macd/adj_close+1)-(signalline/adj_close+1)
    return score

In [169]:
# set weighting when close is lower lower_band or upper upper_band
def bollinger_score(row):
    lower_band = row['bollinger_lower_band']
    upper_band = row['bollinger_upper_band']
    middle_band = row['bollinger']
    adj_close = row['Adj Close']
    if (adj_close>=middle_band):
        if adj_close>=upper_band:
            score = ((adj_close/upper_band)-1)*(-1)
        else:
            score = ((adj_close/((upper_band-middle_band)*0.5+middle_band))-1)*(-1)
    else:
        if (adj_close<=lower_band):
            score = ((adj_close/lower_band)-1)*(-1)
        else:
            score = (adj_close/middle_band)-1
    return score

In [170]:
def rsi_score(row):
    lower_band = 30
    upper_band = 70
    middle_band = (lower_band + upper_band) / 2 
    middle_band1 = 55
    middle_band2 = 45
    outlier_factor = 1.2
    rsi = row['RSI']
    if (rsi>middle_band):
        if rsi>=upper_band:
            score = ((rsi/upper_band)-1)*(-1)*(outlier_factor)
        else:
            if rsi<middle_band1:
                score = 0
            else:
                if rsi<=62.5:
                    score = rsi/middle_band1-1
                else:
                    score = ((62.5-(rsi-62.5))/(middle_band1))-1
    else:
        if (rsi<=lower_band):
            score = ((rsi/lower_band)-1)*(-1)*(outlier_factor)
        else:
            if rsi>middle_band2:
                score = 0
            else:
                if rsi>=37.5:
                    score = (rsi/middle_band2-1)
                else:
                    score = (37.5-(rsi-37.5))/middle_band2-1
    return score

In [171]:
# We should check whether we give the same weighting if it hits above or lower 100/-100
def cci_score(row):
    lower_band = -100
    middle_lower_band = -50
    upper_band = 105
    middle_band = 0
    middle_upper_band = 85
    outlier_factor = 0.25
    outlier_factor1 = 2
    cci=row['RSI']

    if (cci < lower_band):
        score=(cci/lower_band)-1
    else:
        if (cci<=middle_lower_band):
            score = (cci / lower_band)-1
        elif (cci<middle_band):
            score = (((cci-lower_band)/lower_band)+1)*(-1)
        else:
            if (cci>upper_band):
                score = ((cci/upper_band)-1)*(-1)
            else:
                if(cci<=middle_upper_band):
                    score = (cci/upper_band)*outlier_factor
                else:
                    if(cci<upper_band):
                        score=(middle_upper_band-(cci-middle_upper_band))/upper_band-(middle_upper_band-(middle_upper_band-upper_band))/middle_upper_band
                    else:
                        score=0
    
    return score

In [172]:
## TO BE ADDED: DEAD-CAT-BOUNCE SCORE CALC

In [173]:
def relative_score(row):
    r = row
    stock = row['stock_id']

    ema_scr = ema_score(r)
    macd_signalline_scr = macd_signalline_score(r)
    bollinger_scr = bollinger_score(r)
    sma_scr = sma_score(r)
    rsi_scr = rsi_score(r)
    cci_scr = cci_score(r)

    score = ema_scr + (macd_signalline_scr*5) + bollinger_scr + sma_scr + rsi_scr + cci_scr
    print(stock, ema_scr, (macd_signalline_scr*5), bollinger_scr, sma_scr, rsi_scr, cci_scr, score, '\n')
    return score

In [174]:
df_yd['score'] = df_yd.apply(lambda row: relative_score(row), axis=1)
df_yd.sort_values(by=['score'], ascending=False)

AMZN 0.054877464385361785 -0.009031480789706281 0.0003747971553743623 0.03465004643287872 0.10910763955189684 0.14524028613179601 0.33521875286760144 

AAPL 0.06861914676501057 -0.002217787730176557 -0.0069490994285079655 0.05241223423041563 0.08043814674263472 0.15613309983132165 0.348435740410698 

SAP 0.03414706456011096 -0.04248560545479119 -0.030030667635686736 -0.03212566591094379 -0.12568382828253533 0.0848946958874145 -0.11128400683643158 

ACN 0.06858925447794473 0.003418687693841349 -0.009004983137947287 0.048057468465271924 -0.11234081510145995 0.18226955765298053 0.1809891700506313 

VOW.DE 0.055352586441215434 0.00972426228560197 0.015411340389705708 0.00647763783021893 0 0.12780943591638463 0.21477526286312668 



<ipython-input-174-8c16aa146636>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yd['score'] = df_yd.apply(lambda row: relative_score(row), axis=1)


,Open,High,Low,Close,Adj Close,stock_id,ema_25,ema_100,sma_25,sma_100,bollinger,bollinger_lower_band,bollinger_upper_band,macd,signalline,RSI,CCI,dead_cat_bounce,score
date,,,,,,,,,,,,,,,,,,,
2021-07-26,148.27,149.8300,147.70,148.99,148.99,AAPL,142.031054,132.910826,141.570000,130.459414,143.386617,134.738020,152.536980,3.955379,4.021465,65.575902,89.287243,False,0.348436
2021-07-26,3673.17,3712.0800,3647.25,3699.82,3699.82,AMZN,3570.327569,3384.589860,3575.914400,3324.873300,3592.244704,3398.240300,3810.169700,58.926136,65.609107,61.000920,36.860419,False,0.335219
2021-07-26,277.20,277.8000,272.40,277.40,277.40,VOW.DE,276.966493,262.439773,275.614668,275.614986,276.603157,263.143932,286.880888,-0.938148,-1.477650,53.679963,-26.116576,False,0.214775
2021-07-26,318.80,319.7444,316.85,318.98,318.98,ACN,305.961600,286.322925,304.353539,285.336575,308.292785,292.999000,323.973666,7.012579,6.794480,76.553214,117.545048,False,0.180989
2021-07-26,139.09,139.3400,138.26,138.66,138.66,SAP,142.879402,138.161589,143.262400,135.868877,142.952973,135.962543,151.961457,-0.367380,0.810831,35.655772,-94.851858,False,-0.111284
